In [7]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from random import random
from random import uniform
from sklearn.svm import OneClassSVM as OCSVM
import csv
from pathlib import Path
import collections
from numpy import savetxt
import matplotlib.pyplot as plt
import matplotlib.font_manager
from sklearn import svm
from matplotlib.pyplot import figure

In [8]:
fold = 'Fold6/'

In [9]:
dic_DistilBERT = {}           
basepath = Path('/content/drive/My Drive/USP/qualis/Experimentos/DataSets/DBERT-WITH-OTHER-CLASSES/'+ fold)
files_in_basepath = basepath.iterdir()
for item in files_in_basepath:
  if item.is_file():
    arq = open('/content/drive/My Drive/USP/qualis/Experimentos/DataSets/DBERT-WITH-OTHER-CLASSES/'+ fold + item.name, 'r')
    print(item.name)
    mat_embedding = list()
    reader = csv.reader(arq)
    for doc in reader:
      mat_embedding.append([float(i) for i in doc])

    nome_partes = item.name.split('_')
    nome = nome_partes[0] + '_' + nome_partes[2] + '_' + nome_partes[3]
    dic_DistilBERT[nome] = np.array(mat_embedding)
    arq.close()

Sport_DistilBERTML_2000_6000.csv
Drama_DistilBERTML_2000_6000.csv
Opera_DistilBERTML_2000_6000.csv
Design_DistilBERTML_2000_6000.csv
Arson_DistilBERTML_2000_6000.csv
Coal_DistilBERTML_2000_6000.csv
Investigation_DistilBERTML_2000_6000.csv
Media_DistilBERTML_2000_6000.csv
Plant_DistilBERTML_2000_6000.csv
War_DistilBERTML_2000_6000.csv
Dance_DistilBERTML_2000_6000.csv
Law_DistilBERTML_2000_6000.csv
Fire_DistilBERTML_2000_6000.csv
Toy_DistilBERTML_2000_6000.csv
Bonds_DistilBERTML_2000_6000.csv
Metal_DistilBERTML_2000_6000.csv
Loans_DistilBERTML_2000_6000.csv
Parks_DistilBERTML_2000_6000.csv
Diet_DistilBERTML_2000_6000.csv
Arrest_DistilBERTML_2000_6000.csv
Racism_DistilBERTML_2000_6000.csv
Agriculture_DistilBERTML_2000_6000.csv
Terrorism_DistilBERTML_1975_5926.csv
Disaster_DistilBERTML_2000_6000.csv
Immigration_DistilBERTML_2000_6000.csv
Inflation_DistilBERTML_2000_6000.csv
University_DistilBERTML_2000_6000.csv
Election_DistilBERTML_2000_6000.csv
Hospital_DistilBERTML_2000_6000.csv
Cinema_

In [10]:
def VariationalAE():  
  class Sampling(layers.Layer):
      """Uses (z_mean, z_log_var) to sample z."""

      def call(self, inputs):
          z_mean, z_log_var = inputs
          batch = tf.shape(z_mean)[0]
          dim = tf.shape(z_mean)[1]
          epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
          return z_mean + tf.exp(0.5 * z_log_var) * epsilon


  latent_dim = 2

  encoder_inputs = keras.Input(shape=(512,), name='Entrada_Encoder') 
  
  dense1 = layers.Dense(64, activation="relu", name='Dense1')(encoder_inputs)

  z_mean = layers.Dense(latent_dim, name="Z_mean")(dense1)
  z_log_var = layers.Dense(latent_dim, name="Z_log_var")(dense1)
  z = Sampling()([z_mean, z_log_var])
  encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="Encoder")
  #encoder.summary()

  latent_inputs = keras.Input(shape=(latent_dim,),name='Entrada_Decoder')
 
  dense2 = layers.Dense(64, activation="relu", name='Dense2')(latent_inputs)

  decoder_outputs = layers.Dense(512, activation="sigmoid")(dense2)

  decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
  #decoder.summary()


  class VAE(keras.Model):
      def __init__(self, encoder, decoder, **kwargs):
          super(VAE, self).__init__(**kwargs)
          self.encoder = encoder
          self.decoder = decoder

      def train_step(self, data):
          if isinstance(data, tuple):
              data = data[0]
          with tf.GradientTape() as tape:
              z_mean, z_log_var, z = encoder(data)
              reconstruction = decoder(z)
              reconstruction_loss = tf.reduce_mean(
                  keras.losses.mean_squared_error(data, reconstruction)
              )
              reconstruction_loss *= 512
              kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
              kl_loss = tf.reduce_mean(kl_loss)
              kl_loss *= -0.5
              total_loss = reconstruction_loss + kl_loss


          grads = tape.gradient(total_loss, self.trainable_weights)
          self.optimizer.apply_gradients(zip(grads, self.trainable_weights))

          return {
              "loss": total_loss,
              "reconstruction_loss": reconstruction_loss,
              "kl_loss": kl_loss,
          }
          
  vae = VAE(encoder, decoder)

  return vae,encoder

In [ ]:
for epoca in range(1,11):
  for base in dic_DistilBERT.keys(): 

    dataset = dic_DistilBERT[base]
    nome_partes = base.split('_')

    nome = nome_partes[0]
    treino = int(nome_partes[1])
    teste = int(nome_partes[2].replace('.csv',''))

    X = np.array(dataset[:treino])

    vae,encoder = VariationalAE()
    vae.compile(optimizer=keras.optimizers.Adam())
    vae.fit(X, epochs=epoca, batch_size=64)

    data_train,_,_ = encoder.predict(X)
 
    X_test = np.array(dataset[treino:teste])

    data_test,_,_ = encoder.predict(X_test)

    X_outlier = np.array(dataset[teste:])
    
    data_outlier,_,_ = encoder.predict(X_outlier)

    data_total = np.concatenate([data_train,data_test,data_outlier]) 

    name = '/content/drive/My Drive/USP/qualis/Experimentos/DataSets/SVAE/'+ fold + nome + '_VAE_'+str(treino)+'_'+str(teste)+'_'+ str(epoca) +'.csv'
    

    savetxt(name, data_total, delimiter=',')

Streaming output truncated to the last 5000 lines.
Epoch 6/9
32/32 [==============================] - 0s 3ms/step - loss: 4.7276 - reconstruction_loss: 1.7187 - kl_loss: 3.0089
Epoch 7/9
32/32 [==============================] - 0s 3ms/step - loss: 4.2110 - reconstruction_loss: 1.5984 - kl_loss: 2.6125
Epoch 8/9
32/32 [==============================] - 0s 3ms/step - loss: 3.7311 - reconstruction_loss: 1.4220 - kl_loss: 2.3091
Epoch 9/9
32/32 [==============================] - 0s 4ms/step - loss: 3.3924 - reconstruction_loss: 1.3647 - kl_loss: 2.0277
Epoch 1/9
32/32 [==============================] - 1s 3ms/step - loss: 107.4423 - reconstruction_loss: 107.2541 - kl_loss: 0.1882
Epoch 2/9
32/32 [==============================] - 0s 4ms/step - loss: 39.5838 - reconstruction_loss: 33.7151 - kl_loss: 5.8687
Epoch 3/9
32/32 [==============================] - 0s 3ms/step - loss: 14.9532 - reconstruction_loss: 7.4694 - kl_loss: 7.4838
Epoch 4/9
32/32 [==============================] - 0s 3ms/st

In [6]:
basepath = Path('/content/drive/My Drive/USP/qualis/Experimentos/DataSets/SVAE/Fold6/')
files_in_basepath = basepath.iterdir()
count =0
for item in files_in_basepath:
  if item.is_file():
    count+=1
count

0